In [1]:
import torch
from torch import nn
from torch.nn import functional as F
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[ 0.2881,  0.1163,  0.0060,  0.1075, -0.0227,  0.0456, -0.0488,  0.2393,
          0.0442, -0.3043],
        [ 0.2908,  0.1605, -0.0210,  0.0386,  0.0483,  0.1161, -0.1026,  0.2286,
          0.0802, -0.2686]], grad_fn=<AddmmBackward0>)

In [2]:
class MLP(nn.Module):
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        #这样，在类实例化时也可以指定其他函数参数
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X)
        

tensor([[ 0.0790,  0.1703,  0.1375,  0.2858, -0.2195,  0.0323, -0.2555, -0.0512,
         -0.0147,  0.1182],
        [ 0.0907, -0.0534,  0.0676,  0.2651, -0.2690,  0.0506, -0.1389, -0.0880,
         -0.0050, -0.0301]], grad_fn=<AddmmBackward0>)

In [3]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self,X):
        for block in self._modules.values():
            X = block(X)
        return X

In [4]:
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[ 0.0219,  0.0411,  0.1432, -0.0853, -0.1439,  0.1436, -0.3088, -0.1688,
         -0.0110, -0.2088],
        [-0.0794,  0.1205,  0.0150, -0.0765, -0.0897,  0.1437, -0.2706, -0.1196,
         -0.0914, -0.0375]], grad_fn=<AddmmBackward0>)

In [5]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad = False)
        self.linear = nn.Linear(20,20)

    def forward(self,X):
        X = self.linear(X)
        X = F.relu(torch.mm(X,self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [6]:
net = FixedHiddenMLP()
net(X)

tensor(-0.1184, grad_fn=<SumBackward0>)

In [7]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                                 nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)

    def forward(self,X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)
print(chimera)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=20, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=32, bias=True)
      (3): ReLU()
    )
    (linear): Linear(in_features=32, out_features=16, bias=True)
  )
  (1): Linear(in_features=16, out_features=20, bias=True)
  (2): FixedHiddenMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


In [8]:
import torch
from torch import nn
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.2519],
        [-0.2347]], grad_fn=<AddmmBackward0>)

In [9]:
print(net)
print(net.state_dict())
print(net[2].state_dict())

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)
OrderedDict([('0.weight', tensor([[-0.3950,  0.2128,  0.1442,  0.2523],
        [-0.3996, -0.3796, -0.0121,  0.2295],
        [-0.2593,  0.2746,  0.1754, -0.4681],
        [-0.0038,  0.4718, -0.1787, -0.0759],
        [-0.4996,  0.0760,  0.1363, -0.2796],
        [ 0.0221,  0.1584, -0.1604, -0.1997],
        [ 0.4986, -0.2319, -0.3809,  0.2980],
        [ 0.3822,  0.1038, -0.3766,  0.4733]])), ('0.bias', tensor([-0.2535,  0.3367, -0.4663, -0.3031,  0.0270, -0.4074, -0.4898,  0.2912])), ('2.weight', tensor([[-0.0773,  0.0498,  0.1284,  0.1398, -0.1083,  0.3386,  0.2900, -0.1857]])), ('2.bias', tensor([-0.1739]))])
OrderedDict([('weight', tensor([[-0.0773,  0.0498,  0.1284,  0.1398, -0.1083,  0.3386,  0.2900, -0.1857]])), ('bias', tensor([-0.1739]))])


In [10]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(type(net[2].weight))
print(net[2].weight)
print(net[2].weight.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1739], requires_grad=True)
tensor([-0.1739])
<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([[-0.0773,  0.0498,  0.1284,  0.1398, -0.1083,  0.3386,  0.2900, -0.1857]],
       requires_grad=True)
tensor([[-0.0773,  0.0498,  0.1284,  0.1398, -0.1083,  0.3386,  0.2900, -0.1857]])


In [11]:
net[2].weight.grad = None


In [12]:
print(*[(name, param.shape) for name, param in net.named_parameters()])
[(name,param.shape) for name, param in net[0].named_parameters()]

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]

In [13]:
net.parameters()

<generator object Module.parameters at 0x0000016236C202E0>

In [14]:
net.state_dict()['2.bias'].data

tensor([-0.1739])

In [15]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
           nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.4967],
        [-0.4967]], grad_fn=<AddmmBackward0>)

In [16]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [17]:
rgnet[0][1][0].bias.data

tensor([ 0.1469, -0.1553, -0.1937,  0.1439,  0.4210,  0.2566, -0.4011, -0.1438])

In [18]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean = 0, std = 0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
print(net)
net[0].weight.data, net[0].bias.data[0]

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)


(tensor([[ 0.0017,  0.0016, -0.0143,  0.0009],
         [-0.0246, -0.0084, -0.0205,  0.0139],
         [ 0.0096, -0.0134,  0.0074,  0.0004],
         [-0.0251,  0.0127,  0.0018,  0.0026],
         [-0.0021, -0.0031, -0.0261, -0.0022],
         [-0.0012, -0.0098, -0.0129, -0.0126],
         [ 0.0006,  0.0211, -0.0117, -0.0106],
         [ 0.0058,  0.0033, -0.0179, -0.0031]]),
 tensor(0.))

### 将所有参数初始化为给定的常数

In [20]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [21]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,42)
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.3558, -0.5397, -0.4687,  0.2109])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [22]:
def my_init(m):
    if type(m) == nn.Linear:
        # '*'操作是unpacking，把元祖或列表的元素逐个提取出来 
        print('init',*[(name,param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

init weight torch.Size([8, 4])
init weight torch.Size([1, 8])


tensor([[-0.0000,  0.0000,  0.0000,  0.0000],
        [-5.0873,  5.6127,  7.3465,  9.7861]], grad_fn=<SliceBackward0>)

In [23]:
print(net)
[(name, param.shape) for name, param in net.named_parameters()]

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)


[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

In [24]:
[(name,param.shape) for name, param in net.named_parameters()][0]

('0.weight', torch.Size([8, 4]))

In [25]:
print('init',[(name,param.shape) for name, param in net.named_parameters()][0])
print('init',*[(name,param.shape) for name, param in net.named_parameters()][0])

init ('0.weight', torch.Size([8, 4]))
init 0.weight torch.Size([8, 4])


In [26]:
print(net[0].weight.data)
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42 # tensor可以直接这样写访问0,0位置上的元素
net[0].weight.data[0]

tensor([[-0.0000,  0.0000,  0.0000,  0.0000],
        [-5.0873,  5.6127,  7.3465,  9.7861],
        [-0.0000, -0.0000, -0.0000,  9.9452],
        [-7.4317,  0.0000, -0.0000, -0.0000],
        [ 0.0000, -9.5917, -0.0000, -6.5016],
        [-0.0000,  6.0040, -0.0000, -0.0000],
        [-0.0000, -9.9364,  0.0000,  6.5011],
        [ 0.0000, -0.0000,  0.0000,  0.0000]])


tensor([42.,  1.,  1.,  1.])

In [27]:
# 两个shared层共享参数，梯度是它们的梯度之和
shared = nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                   shared,nn.ReLU(),
                   shared, nn.ReLU(),
                   nn.Linear(8,1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


### 延后初始化

In [29]:
import torch
import torch.nn.functional as F
from torch import nn
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, X):
        return X- X.mean()

In [30]:
layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [31]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [32]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-5.1223e-09, grad_fn=<MeanBackward0>)

In [33]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [34]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.5687,  0.7563,  0.2257],
        [-0.8320, -1.0502,  2.6032],
        [-1.2867,  1.4716, -0.2495],
        [ 1.3699, -0.3316,  0.4436],
        [ 0.1907, -1.4614,  0.1801]], requires_grad=True)

In [35]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.0000, 1.9506],
        [0.0000, 0.0000, 2.2542]])

In [36]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[24.1731],
        [ 3.7094]])

### 模型保存

In [38]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [39]:
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [40]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [41]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    def forward(self, x):
         return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [42]:
torch.save(net.state_dict(),'mlp.params')

In [43]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [44]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])